In [36]:
from qdrant_client import QdrantClient, models

In [37]:
client = QdrantClient(url="http://localhost:6333")

In [38]:
if client.collection_exists(collection_name="collection0"):
    client.delete_collection(collection_name="collection0")

client.create_collection(
    collection_name="collection0",
    vectors_config=models.VectorParams(size=3, distance=models.Distance.DOT),
)

client.upsert(
    collection_name="collection0",
    points=models.Batch(
        ids=[1, 2],
        payloads=[
            {
                "dinosaur": "t-rex",
                "diet": [
                    {"food": "leaves", "likes": False},
                    {"food": "meat", "likes": True}
                ]
            },
            {
                "dinosaur": "diplodocus",
                "diet": [
                    {"food": "leaves", "likes": True},
                    {"food": "meat", "likes": False}
                ]
            }
        ],
        vectors=[
            [0.1, 0.1, 0.1],
            [0.2, 0.2, 0.2],
        ],
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [39]:
"""
目标:查询food = "meat" and likes = True的point

分析:
id=1满足条件
    * diet[1]['food'] = 'meat' and diet[1]['likes'] = True
id=2满足条件(与实际目标不符;多个值时,MatchValue只要有一个条件满足就匹配)
    * diet[1]['food'] = 'meat' and diet[0]['likes'] = True
"""
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="diet[].food", match=models.MatchValue(value="meat")
            ),
            models.FieldCondition(
                key="diet[].likes", match=models.MatchValue(value=True)
            ),
        ],
    ),
)

([Record(id=1, payload={'diet': [{'food': 'leaves', 'likes': False}, {'food': 'meat', 'likes': True}], 'dinosaur': 't-rex'}, vector=None, shard_key=None, order_value=None),
  Record(id=2, payload={'diet': [{'food': 'leaves', 'likes': True}, {'food': 'meat', 'likes': False}], 'dinosaur': 'diplodocus'}, vector=None, shard_key=None, order_value=None)],
 None)

In [40]:
"""
目标:查询food = "meat" and likes = True的point

分析:
id=1满足条件
    * diet[1]['food'] = 'meat' and diet[1]['likes'] = True
id=2不满足条件(与实际相符)
    * diet[0]['leaves'] = 'meat'不满足,diet[1]['likes'] = False不满足
"""
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        must=[
            # Nested filters work in the same way as if the nested filter was applied to a single element of the array at a time. Parent document is considered to match the condition if at least one element of the array matches the nested filter.
            models.NestedCondition(
                nested=models.Nested(
                    key="diet",
                    filter=models.Filter(
                        must=[
                            models.FieldCondition(
                                key="food", match=models.MatchValue(value="meat")
                            ),
                            models.FieldCondition(
                                key="likes", match=models.MatchValue(value=True)
                            ),
                        ]
                    ),
                )
            )
        ],
    ),
)

([Record(id=1, payload={'diet': [{'food': 'leaves', 'likes': False}, {'food': 'meat', 'likes': True}], 'dinosaur': 't-rex'}, vector=None, shard_key=None, order_value=None),
  Record(id=2, payload={'diet': [{'food': 'leaves', 'likes': True}, {'food': 'meat', 'likes': False}], 'dinosaur': 'diplodocus'}, vector=None, shard_key=None, order_value=None)],
 None)